<a href="https://colab.research.google.com/github/speacock103/MachineLearningFall2020/blob/master/TitanicNBKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [34]:
import io
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

Grab Titanic Dataset

In [35]:
from google.colab import files
uploaded = files.upload()

titanic_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
titanic_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

Saving test.csv to test (3).csv
Saving train.csv to train (3).csv


Preprocessing

In [36]:
titanic_train = titanic_train.drop("PassengerId", axis=1)
titanic_train = titanic_train.drop("Name", axis=1)
titanic_train = titanic_train.drop("Ticket", axis=1)
titanic_train = titanic_train.drop("Cabin", axis=1)
titanic_test = titanic_test.drop("PassengerId", axis=1)
titanic_test = titanic_test.drop("Name", axis=1)
titanic_test = titanic_test.drop("Ticket", axis=1)
titanic_test = titanic_test.drop("Cabin", axis=1)

sex = {"male": 0, "female": 1}
data = [titanic_train, titanic_test]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(sex)

import random as rd

age_mean = np.mean(titanic_train.Age)
age_std = np.std(titanic_train.Age)
minus = age_mean - age_std
plus = age_mean + age_std

for row in titanic_train.loc[titanic_train.Age.isnull(), 'Age'].index:
    titanic_train.at[row, 'Age'] = rd.randint(int(minus), int(plus))

for row in titanic_test.loc[titanic_test.Age.isnull(), 'Age'].index:
    titanic_test.at[row, 'Age'] = rd.randint(int(minus), int(plus))

port = {"S": 0, "C": 1, "Q": 2}
portdata = [titanic_train, titanic_test]

for dataset in portdata:
    dataset['Embarked'] = dataset['Embarked'].map(port)

Embarkedtop = titanic_train.Embarked.mode()
for row in titanic_train.loc[titanic_train.Embarked.isnull(), 'Embarked'].index:
    titanic_train.at[row, 'Embarked'] = Embarkedtop

Embarkedtoptest = titanic_test.Embarked.mode()
for row in titanic_test.loc[titanic_test.Embarked.isnull(), 'Embarked'].index:
    titanic_test.at[row, 'Embarked'] = Embarkedtoptest

Faretop = titanic_train.Fare.mode()
for row in titanic_train.loc[titanic_train.Fare.isnull(), 'Fare'].index:
    titanic_train.at[row, 'Fare'] = Faretop
  
Faretoptest = titanic_test.Fare.mode()
for row in titanic_test.loc[titanic_test.Fare.isnull(), 'Fare'].index:
    titanic_test.at[row, 'Fare'] = Faretoptest

titanic_trainrev = titanic_train.drop("Survived", axis=1)
survival_train = titanic_train["Survived"]

# Naive Bayes with Titanic Data Set

fitting of training data to model

In [37]:
titanicnb = GaussianNB()
titanicnb.fit(titanic_trainrev, survival_train)

GaussianNB(priors=None, var_smoothing=1e-09)

5-fold cross validation

In [38]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

scores = cross_validate(titanicnb, titanic_trainrev, survival_train, cv=5, scoring = scoring)
print (scores)

{'fit_time': array([0.00438929, 0.00242662, 0.00246501, 0.00238609, 0.00248694]), 'score_time': array([0.00555611, 0.00384712, 0.00369906, 0.00402355, 0.00390434]), 'test_accuracy': array([0.76536313, 0.79213483, 0.80337079, 0.78651685, 0.79775281]), 'test_precision': array([0.69014085, 0.73134328, 0.73913043, 0.74193548, 0.7704918 ]), 'test_recall': array([0.71014493, 0.72058824, 0.75      , 0.67647059, 0.68115942]), 'test_f1_score': array([0.7       , 0.72592593, 0.74452555, 0.70769231, 0.72307692])}


# K-Nearest Neighbors with Titanic Data Set

In [50]:
from google.colab import files
uploaded = files.upload()

titanic_testy = pd.read_csv(io.BytesIO(uploaded['gender_submission.csv']))

Saving gender_submission.csv to gender_submission.csv


In [63]:
def knn_distance(trainx,testx,k):
    distances = -2 * trainx@testx.T + np.sum(testx**2,axis=1) + np.sum(trainx**2,axis=1)[:, np.newaxis]
    distances[distances < 0] = 0
    distances = distances**.5
    indices = np.argsort(distances, 0)
    distances = np.sort(distances,0)
    return indices[0:k,:], distances[0:k,:]


def knn_prediction(trainx,trainy,testx,k):
    indices, distances = knn_distance(trainx,testx,k)
    trainy = trainy.flatten()
    rows, columns = indices.shape
    predictions = list()
    for j in range(columns):
        temp = list()
        for i in range(rows):
            cell = indices[i][j]
            temp.append(trainy[cell])
        predictions.append(max(temp,key=temp.count))
    predictions=np.array(predictions)
    return predictions

def knn_accuracy(testy,predictions):
    x=testy.flatten()==predictions.flatten()
    grade=np.mean(x)
    return np.round(grade*100,2)

from tqdm import tqdm_notebook

titanic_trainrevarr = titanic_trainrev.values

survival_trainarr = survival_train.values

titanic_testarr = titanic_test.values

titanic_testyrev = titanic_testy.drop("PassengerId", axis=1)
titanic_testyarr = titanic_testyrev.values

accuracies = []
k_values = [i for i in range(1,200)]
for k in tqdm_notebook(k_values):
    prediction = knn_prediction(titanic_trainrevarr, survival_trainarr, titanic_testarr, k)
    accuracy = knn_accuracy(titanic_testyarr, prediction)
    accuracies.append(accuracy)

bestk = k_values[accuracies.index(max(accuracies))]
print (bestk)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



20


In [65]:
accuracies[19]

66.03